In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LeakyReLU
from imblearn.over_sampling import SMOTE
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from sklearn.feature_selection import SelectKBest, f_classif

In [33]:
df = pd.read_csv('/content/StudentPerformanceFactors.csv')

## Kode tersebut menyandikan (encode) kolom-kolom kategori dalam dataframe menggunakan LabelEncoder, sehingga nilai kategori diubah menjadi angka numerik untuk keperluan pemrosesan oleh model machine learning.

In [34]:
# Preprocessing
# Menyandikan (encode) kolom-kolom kategori
kolom_kategori = ['Parental_Involvement', 'Access_to_Resources', 'Extracurricular_Activities',
                  'Motivation_Level', 'Internet_Access', 'Family_Income', 'Teacher_Quality',
                  'School_Type', 'Peer_Influence', 'Learning_Disabilities',
                  'Parental_Education_Level', 'Distance_from_Home', 'Gender']

label_encoders = {}
for kolom in kolom_kategori:
    label_encoders[kolom] = LabelEncoder()
    df[kolom] = label_encoders[kolom].fit_transform(df[kolom].astype(str))

## Kode tersebut menggunakan SelectKBest untuk memilih 10 fitur terbaik dari dataset berdasarkan uji statistik ANOVA (f_classif), yang mengevaluasi hubungan antara setiap fitur dan target. Dengan fit_transform, fitur-fitur yang paling berpengaruh terhadap variabel target (y) dipilih dan diubah menjadi subset baru X_new. Teknik ini membantu meningkatkan kinerja model dengan hanya mempertahankan fitur yang paling relevan, mengurangi dimensi, dan menghindari overfitting dari fitur yang kurang berkontribusi.

In [35]:
# Memilih fitur terbaik menggunakan SelectKBest
selector = SelectKBest(score_func=f_classif, k=10)  # Memilih 10 fitur terbaik
X_new = selector.fit_transform(X, y)

## Kode tersebut menangani nilai kosong (missing values) dalam dataframe df dengan cara mengisi nilai kosong di setiap kolom menggunakan modus (nilai yang paling sering muncul) dari kolom tersebut. df.mode().iloc[0] menghitung modus untuk setiap kolom, lalu fillna() mengisi nilai kosong dengan nilai tersebut secara langsung di dataframe (inplace=True). Pendekatan ini memastikan tidak ada nilai kosong dalam dataset sebelum digunakan dalam model.

In [36]:
# Menangani nilai kosong (missing values)
df.fillna(df.mode().iloc[0], inplace=True)

## Kode tersebut memisahkan fitur (X) dan target (y) dalam dataset. X diisi dengan semua kolom kecuali kolom 'Motivation_Level' (yang merupakan target) menggunakan df.drop('Motivation_Level', axis=1), sedangkan y hanya diisi dengan kolom 'Motivation_Level' yang menjadi variabel target. Hal ini dilakukan untuk mempersiapkan data yang akan digunakan dalam pelatihan model, di mana X berisi input atau fitur dan y adalah label atau output yang akan diprediksi.








In [37]:
# Memisahkan fitur dan target
X = df.drop('Motivation_Level', axis=1)
y = df['Motivation_Level']

In [38]:
# Melakukan normalisasi pada fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [39]:
# Mengubah target menjadi one-hot encoding
y_encoded = to_categorical(y)

In [40]:
# Membagi dataset menjadi data training dan validasi
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [41]:
# Membangun model Sequential
model = Sequential()

In [42]:
# Lapisan Input
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.10))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [56]:
# Lapisan Input dengan Dropout dan Regularisasi
model.add(Dense(512, input_dim=X_train.shape[1], kernel_regularizer=l2(0.001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))  # Dropout lebih besar untuk mencegah overfitting

# Lapisan Tersembunyi
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.003))  # Dropout ditingkatkan menjadi 0.3

# Lapisan Tersembunyi
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.003))  # Dropout pada 0.3 untuk menjaga regulasi

In [57]:
# Lapisan Output
model.add(Dense(3, activation='softmax'))

In [58]:
# Optimizer dengan learning rate lebih kecil
optimizer = SGD(learning_rate=0.0001, momentum=0.10)

In [59]:
optimizer = Adam(learning_rate=0.0001)

In [60]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [61]:
# Kompilasi model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [62]:
# Early stopping dengan patience lebih rendah
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [65]:
# Melatih model
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_val, y_val))

Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8373 - loss: 0.4765 - val_accuracy: 0.5431 - val_loss: 1.3273
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8341 - loss: 0.4787 - val_accuracy: 0.5356 - val_loss: 1.3424
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8521 - loss: 0.4519 - val_accuracy: 0.5431 - val_loss: 1.3374
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8476 - loss: 0.4642 - val_accuracy: 0.5356 - val_loss: 1.3193
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8453 - loss: 0.4557 - val_accuracy: 0.5356 - val_loss: 1.3558
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8435 - loss: 0.4697 - val_accuracy: 0.5287 - val_loss: 1.3485
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8513 - loss: 0.4548 - val_accuracy: 0.5310 - val_loss: 1.3447
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.8466 - loss: 0.4647 - val_accuracy: 0.5408 - v

In [66]:
# Menampilkan hasil akurasi training dan validasi
training_accuracy = history.history['accuracy'][-1] * 100
validation_accuracy = history.history['val_accuracy'][-1] * 100
print(f"Akurasi Training: {training_accuracy:.2f}%")
print(f"Akurasi Validasi: {validation_accuracy:.2f}%")

Akurasi Training: 84.45%
Akurasi Validasi: 54.08%
